# Analise preliminar de linguagem nos discursos do Parlamento

## 1.Importar dados

Começamos por importar algumas libraries necessárias. 

In [1]:
import json
import os
import spacy
import nltk


from spacy.lang.pt import Portuguese
from nltk.stem import RSLPStemmer
from tqdm import tqdm

E extraimos um numero limitado de discursos:

In [2]:
JSON_FILES_TO_LOAD = 30
FILES_DIRECTORY = "data/debates/"

parties = []
speakers = []
contents = []

for file_name in os.listdir(FILES_DIRECTORY)[:JSON_FILES_TO_LOAD]:
    with open(FILES_DIRECTORY + file_name) as file:
        json_data = json.load(file)
        for number, intervention in json_data['intervenções'].items():
            speakers.append(intervention["orador"])
            parties.append(intervention["partido"])
            contents.append(intervention["discurso"])

Vamos visualizar um exemplo: 

In [3]:
indice = 899
print("Orador: ", speakers[indice])
print("Partido: ", parties[indice])
print("Discurso: ", contents[indice])

Orador:  Cristóvão Simão Ribeiro
Partido:  PSD
Discurso:  Sr. Presidente, Srs. Membros do Governo, Sr.as Deputadas e Srs. Deputados: Dirijo um cumprimento especial aos estudantes de medicina que se encontram a assistir à sessão. Vou citar um ex-Primeiro-Ministro e começar por dizer o seguinte: quando queremos ser coerentes não podemos ser originais todos os dias. Este é um assunto de tal maneira debatido e replicado nesta Câmara que vou dizer exatamente aquilo que disse nas últimas três ou quatro vezes que este assunto foi aqui trazido repetidamente desde que os senhores são Governo e que a esquerda parlamentar os apoia. Vou dizê-lo em quatro notas particulares. Primeira nota: é evidentemente lesivo e um autêntico defraudar de expetativas, quer para os estudantes, quer para as suas famílias e, ainda pior, altamente lesivo para o Serviço Nacional de Saúde aquilo que é o desperdício de rios de dinheiro em formação médica sem depois haver uma consequência na formação médica especializada 

Quantos recolhemos no total? 

In [4]:
print(f"Oradores: {len(list(set(speakers)))}")
print(f"Partidos: {len(list(set(parties)))}")
print(f"Discursos: {len(contents)}")

Oradores: 364
Partidos: 8
Discursos: 6988


## 2.Limpeza de dados

- **De palavras para tokens limpos**

Criamos a função `tokenize`,  uma função que limpa o texto de acordo com um numero de regras (é texto? é um numero? é pontuação?).

Downloadamos os recursos necessários:

In [5]:
spacy.load('pt')
parser = Portuguese()

E criamos a função 

In [6]:
def tokenize(text):
    tokenized_text = []
    tokens = parser(text)
    for token in tokens:
        if token.is_digit:
            continue
        if token.is_space:
            continue
        if token.is_punct:
            continue
        if token.is_stop:
            continue
        #if token.is_title:
        #    continue
        if token.is_alpha:
            tokenized_text.append(token.lower_)
    return tokenized_text

- **limpar a lista com stopwords**

Importamos a lista de `stopwords` que se encontrão no ficheiro `data/nlp/stopwords.txt`

In [7]:
with open("data/nlp/stopwords.txt") as file: 
    custom_stopwords = file.readlines()
custom_stopwords = [line.strip() for line in custom_stopwords] 

- **stemmer**

Criamos um primeiro stemmer baseado [neste](https://www.nltk.org/_modules/nltk/stem/rslp.html) algoritmo. 

Loadamos os dados:

In [8]:
nltk.download('rslp')
st = RSLPStemmer()

[nltk_data] Downloading package rslp to
[nltk_data]     /Users/duarteocarmo/nltk_data...
[nltk_data]   Package rslp is already up-to-date!


E criamos a função

In [9]:
def stem_rslp(tokens):
    stemmed = []
    for token in tokens:
        stemmed.append(st.stem(word))
    return stemmed

- **lemmatizer**

Criamos um stemmer alternativo baseado num modelo de noticias da library `spacy`.

Loadamos os dados

In [10]:
spacy_pt = spacy.load("pt_core_news_sm")

E criamos a função:

In [11]:
def lemmatize_spacy(tokens):
    text = " ".join(tokens)
    lemma = []
    for token in spacy_pt(text):
        if token.lemma_:
            lemma.append(token.lemma_)
        else:
            lemma.append(token)
    return lemma

- **Função de limpeza de texto**

Estamos finalmente prontos para criar uma função que recebe um discurso e limpa o totalmente.

In [12]:
def prepare_text(text):
    tokens = tokenize(text)
    tokens = lemmatize_spacy(tokens)
    #tokens = stem_rslp(tokens)
    tokens = [token for token in tokens if token not in custom_stopwords]
    return tokens

Vamos testar com o mesmo exemplo de acima:

In [13]:
texto_original = contents[indice]
tokens_limpos = prepare_text(texto_original)
texto_limpo = " ".join(tokens_limpos)

print(f"Texto Original:\n{texto_original}")
print("- " * 10)
print(f"Texto Limpo:\n{tokens_limpos}")

Texto Original:
Sr. Presidente, Srs. Membros do Governo, Sr.as Deputadas e Srs. Deputados: Dirijo um cumprimento especial aos estudantes de medicina que se encontram a assistir à sessão. Vou citar um ex-Primeiro-Ministro e começar por dizer o seguinte: quando queremos ser coerentes não podemos ser originais todos os dias. Este é um assunto de tal maneira debatido e replicado nesta Câmara que vou dizer exatamente aquilo que disse nas últimas três ou quatro vezes que este assunto foi aqui trazido repetidamente desde que os senhores são Governo e que a esquerda parlamentar os apoia. Vou dizê-lo em quatro notas particulares. Primeira nota: é evidentemente lesivo e um autêntico defraudar de expetativas, quer para os estudantes, quer para as suas famílias e, ainda pior, altamente lesivo para o Serviço Nacional de Saúde aquilo que é o desperdício de rios de dinheiro em formação médica sem depois haver uma consequência na formação médica especializada que, justamente, esses estudantes e esses 

Parece produzir um resultado *aceitavel*.

## 3.Detecção de tópicos

Começamos por importatr algumas bibliotecas importantes como `gensim`. Com ela vamos usar um modelo `Latent Dirichlet allocation`.

In [14]:
import gensim # lda e outros modelos
import pickle # para guardar modelos
from gensim import corpora # para criar um corpora instance

Começamos por criar uma lista onde cada elemento é um lista que contém os "tokens" de um discurso de um deputado. 

In [15]:
clean_text_data = []
for i in tqdm(range(len(contents)), ascii=True):
    if speakers[i] == "Presidente (Jorge Lacão)":
        continue
    else:
        clean_text_data.append(prepare_text(contents[i]))

100%|##########| 6988/6988 [03:58<00:00, 14.35it/s]


De seguida, criamos dois elementos essenciais para a criação de um modelo LDA: 
- o dicionario de termos 
- e o corpus de "bag of words"

In [16]:
dictionnary = corpora.Dictionary(clean_text_data)
corpus = [dictionnary.doc2bow(text) for text in clean_text_data]

Guardamos o corpus e o modelo no directorio `nlp_models`

In [17]:
pickle.dump(corpus, open('nlp_models/corpus.pkl', 'wb'))
dictionnary.save('nlp_models/dictionnary.gensim')

Uma das desvantagens do modelo LDA é que temos de ser nós a definir o numero de topicos. Neste caso, escolhemos 20. 

In [18]:
NUM_TOPICS = 20

Podemos agora criar e guardar o modelo. 

In [19]:
ldamodel = gensim.models.ldamodel.LdaModel(corpus, 
                                          num_topics = NUM_TOPICS, 
                                          id2word=dictionnary, 
                                          passes=15)
ldamodel.save('nlp_models/model5.gensim')

Vamos inspecionar os topicos de uma forma simples: 

In [20]:
topics = ldamodel.print_topics(num_words=9)
for topic in topics:
    print(f"Tópico #{topic[0]} inclui: {topic[1]}")

Tópico #0 inclui: 0.054*"social" + 0.029*"trabalhador" + 0.024*"coletiva" + 0.022*"segurança" + 0.021*"direito" + 0.020*"governar" + 0.016*"reformar" + 0.016*"contratação" + 0.016*"salário"
Tópico #1 inclui: 0.034*"lembrar" + 0.016*"parecer" + 0.015*"negócio" + 0.015*"mutilação" + 0.014*"feminino" + 0.013*"comunicação" + 0.013*"mulher" + 0.013*"alves" + 0.012*"genital"
Tópico #2 inclui: 0.029*"agricultura" + 0.024*"produção" + 0.023*"setor" + 0.021*"pequeno" + 0.020*"alimentar" + 0.019*"produto" + 0.018*"florestar" + 0.014*"rural" + 0.014*"região"
Tópico #3 inclui: 0.060*"deputar" + 0.051*"concluir" + 0.024*"direito" + 0.024*"caducidade" + 0.014*"israel" + 0.013*"mulher" + 0.013*"armar" + 0.011*"liberdade" + 0.011*"militar"
Tópico #4 inclui: 0.035*"governar" + 0.017*"ambientar" + 0.015*"verde" + 0.015*"população" + 0.014*"português" + 0.012*"deputar" + 0.012*"espanha" + 0.011*"d" + 0.010*"central"
Tópico #5 inclui: 0.018*"escola" + 0.016*"público" + 0.014*"governar" + 0.012*"património